In [1]:
import sys

data = sys.stdin.read().strip().split()
# data bevat nu alle tokens van invoer
it = iter(data)

# voorbeeld als eerste token een getal N is
try:
    print(next(it))
except:
    print("no input")

no input


#### 📌 Snelle input + automatische type‑conversie

Dit helpt bij veel testcases met grote data.

**Gebruik:** Voor snelle parsing van grote inputdata bij veel testcases.  

**Wat gebeurt er:** Alle input wordt ingelezen als bytes, opgesplitst, en door een iterator gehaald; `ni()` en `ns()` geven respectievelijk int of string.

In [2]:
import sys

data = sys.stdin.buffer.read().split()
it = iter(data)

def ni():
    return int(next(it))

def ns():
    return next(it).decode()

# Gebruik ni() en ns() voor snelle parsing zonder veel boilerplate.

#### 📌 Combinaties & Permutaties (built-in)

Handig als je geen handmatige backtrack wilt.

**Gebruik:** Snel alle combinaties of permutaties genereren zonder handmatige backtracking.  

**Wat gebeurt er:** `itertools.combinations` genereert subsets van vaste grootte; `itertools.permutations` genereert alle volgordes.

In [3]:
import itertools

for comb in itertools.combinations([1, 2, 3, 4], 2):
    print(comb)

for perm in itertools.permutations([1, 2, 3]):
    print(perm)

(1, 2)
(1, 3)
(1, 4)
(2, 3)
(2, 4)
(3, 4)
(1, 2, 3)
(1, 3, 2)
(2, 1, 3)
(2, 3, 1)
(3, 1, 2)
(3, 2, 1)


#### 📌 Bitmask‑subset iteratie

**Gebruik:** Alle subsets genereren van een set, handig voor combinatorische problemen.  

**Wat gebeurt er:** Loop over alle mogelijke bitmasks; elk mask representeert een subset van indices waar de bit 1 staat.

Handig bij combinatorische problemen:

In [4]:
def subsets_mask(n):
    for mask in range(1 << n):
        yield [i for i in range(n) if mask & (1 << i)]

In [5]:
# Demo for subsets_mask
print("All subsets of {0,1,2} as index lists:")
print(list(subsets_mask(3)))

All subsets of {0,1,2} as index lists:
[[], [0], [1], [0, 1], [2], [0, 2], [1, 2], [0, 1, 2]]


#### 📌 Recursie met memo (cache)

**Gebruik:** Vermijden van dubbele berekeningen in recursieve functies (Fibonacci, DP).  

**Wat gebeurt er:** `lru_cache` slaat eerdere resultaten op; functie rekent alleen nieuwe waarden uit.

In [6]:
from functools import lru_cache

@lru_cache(None)
def f(x):
    if x < 2:
        return x
    return f(x - 1) + f(x - 2)

In [7]:
# Demo for memoized Fibonacci f(x)
print("First 10 Fibonacci numbers:")
print([f(i) for i in range(10)])

First 10 Fibonacci numbers:
[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]


### Graphs

#### 📌 DFS & BFS voor grafen

Velden als mijnenveld, sokoban of kortste pad komen vaak voor.

In [8]:
from collections import deque

# BFS voorbeeld (kortste pad op een grid of graf)
def bfs(start, neighbors):
    visited = {start}
    q = deque([start])
    while q:
        node = q.popleft()
        for nxt in neighbors(node):
            if nxt not in visited:
                visited.add(nxt)
                q.append(nxt)
    return visited

# DFS recursief
def dfs(node, neighbors, visited=None):
    if visited is None: visited = set()
    visited.add(node)
    for nxt in neighbors(node):
        if nxt not in visited:
            dfs(nxt, neighbors, visited)
    return visited

In [9]:
# Demo for bfs and dfs
# Simple undirected graph: 0-1-2 and 1-3
adj = {
    0: [1],
    1: [0, 2, 3],
    2: [1],
    3: [1],
}

print("BFS from 0:", bfs(0, lambda u: adj[u]))
print("DFS from 0:", dfs(0, lambda u: adj[u]))

BFS from 0: {0, 1, 2, 3}
DFS from 0: {0, 1, 2, 3}


#### 📌 Dijkstra voor gewogen grafen

Als een opgave kortste paden met gewichten vraagt.

In [10]:
import heapq

def dijkstra(adj, start):
    dist = {start: 0}
    pq = [(0, start)]
    while pq:
        d, u = heapq.heappop(pq)
        if d > dist[u]:
            continue
        for v, w in adj[u]:
            nd = d + w
            if v not in dist or nd < dist[v]:
                dist[v] = nd
                heapq.heappush(pq, (nd, v))
    return dist

In [11]:
# Demo for dijkstra with traversal order
# Weighted directed graph: 0 -> 1 (2), 0 -> 2 (5), 1 -> 2 (1)
adj = {
    0: [(1, 2), (2, 5)],
    1: [(2, 1)],
    2: [],
}

# Small wrapper that records the pop order from the priority queue

def dijkstra_with_trace(adj, start):
    dist = {start: 0}
    pq = [(0, start)]
    pop_order = []
    while pq:
        d, u = heapq.heappop(pq)
        pop_order.append((u, d))
        if d > dist[u]:
            continue
        for v, w in adj[u]:
            nd = d + w
            if v not in dist or nd < dist[v]:
                dist[v] = nd
                heapq.heappush(pq, (nd, v))
    return dist, pop_order

distances, order = dijkstra_with_trace(adj, 0)
print("Pop order (node, tentative distance):", order)
print("Final shortest distances from 0:", distances)

Pop order (node, tentative distance): [(0, 0), (1, 2), (2, 3), (2, 5)]
Final shortest distances from 0: {0: 0, 1: 2, 2: 3}


#### 📌 9) Grid‑BFS + richtingenbord

Nuttig voor labyrinth/grid‑problemen:

**Gebruik:** Kortste pad of bereikbaarheid in een grid of labyrinth.  

**Wat gebeurt er:** BFS loopt over de grid, houdt afstand bij in een dictionary en negeert muren of reeds bezochte cellen.

In [12]:
from collections import deque

dirs = [(1, 0), (-1, 0), (0, 1), (0, -1)]

def bfs_grid(start, grid):
    R, C = len(grid), len(grid[0])
    q = deque([start])
    dist = {start: 0}
    while q:
        r, c = q.popleft()
        for dr, dc in dirs:
            nr, nc = r + dr, c + dc
            if 0 <= nr < R and 0 <= nc < C and (nr, nc) not in dist and grid[nr][nc] != '#':
                dist[(nr, nc)] = dist[(r, c)] + 1
                q.append((nr, nc))
    return dist

#### 📌 Shortest path (BFS voor unweighted)

**Gebruik:** Kortste pad in een ongewogen graf.  

**Wat gebeurt er:** BFS houdt afstand bij vanaf startknoop; elke nieuwe knoop wordt toegevoegd met afstand +1.

In [13]:
from collections import deque

def shortest_unweighted(adj, start):
    q = deque([start])
    dist = {start: 0}
    while q:
        u = q.popleft()
        for v in adj[u]:
            if v not in dist:
                dist[v] = dist[u] + 1
                q.append(v)
    return dist

#### 📌 Union‑Find (disjoint set)

Handig voor connectiviteitsproblemen:

**Gebruik:** Connectiviteitsproblemen, componentdetectie of cyclusdetectie in grafen.  
**Wat gebeurt er:** `find` zoekt de root van een element (met path compression), `union` verbindt twee sets.

In [14]:
class UF:
    def __init__(self, n):
        self.p = list(range(n))

    def find(self, x):
        if self.p[x] != x:
            self.p[x] = self.find(self.p[x])
        return self.p[x]

    def union(self, x, y):
        rx, ry = self.find(x), self.find(y)
        if rx != ry:
            self.p[ry] = rx

In [15]:
# Demo for Union-Find (disjoint set)
# We create 5 elements (0..4) and unite some of them.
uf = UF(5)

# Unions: {0,1,2} become one component, {3,4} another
uf.union(0, 1)
uf.union(1, 2)
uf.union(3, 4)

# Show representative (root) for each element
roots = [uf.find(i) for i in range(5)]
print("Roots for elements 0..4:", roots)

# Check if pairs are in the same component
same_01 = uf.find(0) == uf.find(1)
same_23 = uf.find(2) == uf.find(3)
print("0 and 1 same component?", same_01)
print("2 and 3 same component?", same_23)

Roots for elements 0..4: [0, 0, 0, 3, 3]
0 and 1 same component? True
2 and 3 same component? False


#### 📌 Topologische sort

Als er een dependency‑graaf is:

**Gebruik:** Taken met afhankelijkheden, zoals projecten of opdrachten.  
**Wat gebeurt er:** BFS-achtige methode die knopen met indegree 0 in volgorde verwerkt; levert een topologische volgorde voor een DAG.

In [16]:
from collections import deque

def topo_sort(n, adj):
    indeg = [0] * n
    for u in range(n):
        for v in adj[u]:
            indeg[v] += 1
    q = deque([i for i in range(n) if indeg[i] == 0])
    res = []
    while q:
        u = q.popleft()
        res.append(u)
        for v in adj[u]:
            indeg[v] -= 1
            if indeg[v] == 0:
                q.append(v)
    return res

In [17]:
# Demo for topo_sort
# DAG: 0 -> 1, 0 -> 2, 1 -> 3, 2 -> 3
n = 4
adj = {
    0: [1, 2],
    1: [3],
    2: [3],
    3: [],
}

order = topo_sort(n, adj)
print("Topological order:", order)

Topological order: [0, 1, 2, 3]


### Mathematical / Number theory

#### 📌 Sieve van Eratosthenes (snelle priemtests)

Als je met primes moet werken.

**Gebruik:** Snel alle priemgetallen ≤ n vinden.  
**Wat gebeurt er:** Een lijst houdt bij welke getallen prime zijn; veelvouden van elke prime worden uitgesloten.

In [18]:
def sieve(n):
    isprime = [True] * (n + 1)
    isprime[0] = isprime[1] = False
    for i in range(2, int(n ** 0.5) + 1):
        if isprime[i]:
            for j in range(i * i, n + 1, i):
                isprime[j] = False
    return [i for i, p in enumerate(isprime) if p]

In [19]:
# Demo for sieve
# List all primes up to 50
print("Primes up to 50:")
print(sieve(50))

Primes up to 50:
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]


#### 📌 Nim‑winnende zetten (voorbeeld uit VPW 2017)

**Gebruik:** Speltheorie – alle zetten vinden die naar een winnende positie leiden.  
**Wat gebeurt er:** Nim-sum wordt berekend, en voor elke stapel wordt gecontroleerd of aanpassen leidt tot een winnende configuratie.

In [20]:
def winning_moves(piles):
    xor_val = 0
    for p in piles:
        xor_val ^= p
    moves = []
    for i, p in enumerate(piles):
        target = p ^ xor_val
        if target < p:
            new = piles.copy()
            new[i] = target
            moves.append(new)
    return sorted(moves)

In [21]:
# Demo for winning_moves (Nim)
# Example piles configuration
piles = [3, 4, 5]
print("Starting piles:", piles)

moves = winning_moves(piles)
print("Winning moves (each list is a new piles configuration):")
for m in moves:
    print(m)

Starting piles: [3, 4, 5]
Winning moves (each list is a new piles configuration):
[1, 4, 5]


#### 📌 Ontbrekend element in reeks (VPW 2017)

**Gebruik:** Vind het ontbrekende getal in een rij 1..n.  
**Wat gebeurt er:** Som van 1..n berekend en verschil genomen met huidige som om het ontbrekende element te vinden.

In [22]:
def missing_number(arr):
    n = len(arr) + 1
    full_sum = n * (n + 1) // 2
    curr_sum = sum(arr)
    missing = full_sum - curr_sum
    return missing if 1 <= missing <= max(arr) else None

In [23]:
# Demo for missing_number
# Array from 1..n with one number missing
arr = [1, 2, 4, 5]
print("Array:", arr)
print("Missing number:", missing_number(arr))

Array: [1, 2, 4, 5]
Missing number: 3


#### 📌 Prime factorisatie (snelle trial)

**Gebruik:** Alle priemfactoren van een getal vinden.  
**Wat gebeurt er:** Iteratief delen door priemgetallen; elke factor wordt toegevoegd aan de lijst; eventuele resterende n > 1 is ook een factor.

In [24]:
def prime_factors(n):
    i = 2
    res = []
    while i * i <= n:
        while n % i == 0:
            res.append(i)
            n //= i
        i += 1
    if n > 1:
        res.append(n)
    return res

In [25]:
# Demo for prime_factors
n = 84
print("Number:", n)
print("Prime factors:", prime_factors(n))

Number: 84
Prime factors: [2, 2, 3, 7]


## Dynamic Programming / Optimization

#### 📌 Dynamische programmering — voorbeeld sub-string of knapsack

Makkelijk aan te passen naar echte contest-vragen.

**Gebruik:** Optimalisatieproblemen zoals knapsack, substrings of subsequences.  
**Wat gebeurt er:** `dp[i][j]` houdt de maximale waarde bij voor de eerste i items en capaciteit j; voor elk item wordt gekeken of opnemen beter is dan niet opnemen.

In [26]:
# klassieke knapsack (waarden en gewichten)
def knapsack(weights, values, W):
    n = len(weights)
    dp = [[0] * (W + 1) for _ in range(n + 1)]
    for i in range(1, n + 1):
        w, v = weights[i - 1], values[i - 1]
        for j in range(W + 1):
            dp[i][j] = dp[i - 1][j]
            if j >= w:
                dp[i][j] = max(dp[i][j], dp[i - 1][j - w] + v)
    return dp[n][W]

In [27]:
# Demo for knapsack
weights = [2, 3, 4]
values = [4, 5, 6]
W = 5
print("Weights:", weights)
print("Values:", values)
print("Capacity:", W)
print("Maximum value achievable:", knapsack(weights, values, W))

Weights: [2, 3, 4]
Values: [4, 5, 6]
Capacity: 5
Maximum value achievable: 9


#### 📌 Sliding window / two pointers

Voor array‑problemen met subranges:

**Gebruik:** Problemen met subarrays, limieten of maximale sommen.  
**Wat gebeurt er:** Twee pointers schuiven over array; houd de som bij en pas aan wanneer limiet overschreden wordt.

In [28]:
def max_subarray_sum_at_most_k(arr, k):
    left = 0
    curr = 0
    best = 0
    for right, x in enumerate(arr):
        curr += x
        while curr > k:
            curr -= arr[left]
            left += 1
        best = max(best, curr)
    return best

In [29]:
# Demo for max_subarray_sum_at_most_k
arr = [1, 2, 3, 4, 5]
k = 7
print("Array:", arr)
print("k:", k)
print("Max subarray sum <= k:", max_subarray_sum_at_most_k(arr, k))

Array: [1, 2, 3, 4, 5]
k: 7
Max subarray sum <= k: 7


### Combinatorial

#### 📌 Backtracking / Permutaties

Handig bij puzzel-achtige opgaven (bijvoorbeeld sokoban-achtige puzzles).

**Gebruik:** Voor het genereren van alle mogelijke volgordes, combinaties of puzzeloplossingen zoals Sokoban of Sudoku.  
**Wat gebeurt er:** Recursief wordt elk element gekozen en toegevoegd aan het huidige pad; wanneer alle keuzes gebruikt zijn, wordt het pad “yielded”.

Gebruik dit ook om combinaties of plaatsen van objecten uit te proberen.

In [30]:
def backtrack(path, choices):
    if not choices:
        yield path
    else:
        for i, c in enumerate(choices):
            yield from backtrack(path + [c], choices[:i] + choices[i + 1:])

In [31]:
for perm in backtrack([], [1, 2, 3]):
     print(perm)

[1, 2, 3]
[1, 3, 2]
[2, 1, 3]
[2, 3, 1]
[3, 1, 2]
[3, 2, 1]


## String manipulation / Other

#### 📌 Frequency count of characters in a string

In [32]:
def char_frequency(s):
    freq = {}
    for c in s:
        freq[c] = freq.get(c, 0) + 1
    return freq

In [33]:
# Demo for kmp_search
text = "ababcabcabababd"
print("Text:", text)
freq = char_frequency(text)
print(freq)

Text: ababcabcabababd
{'a': 6, 'b': 6, 'c': 2, 'd': 1}


#### 📌 Simple substring search (brute force)

In [34]:
def find_substring(text, pattern):
    n, m = len(text), len(pattern)
    for i in range(n - m + 1):
        if text[i:i + m] == pattern:
            return i
    return -1

In [35]:
# Demo for kmp_search
text = "ababcabcabababd"
pattern = "ababd"
print("Text:", text)
print("Pattern:", pattern)
idx = find_substring(text, pattern)
print("Pattern found at index:", idx)

Text: ababcabcabababd
Pattern: ababd
Pattern found at index: 10


#### 📌 KMP algorithm for substring search

In [36]:
def kmp_search(text, pattern):
    def build_lps(pattern):
        lps = [0] * len(pattern)
        length = 0
        i = 1
        while i < len(pattern):
            if pattern[i] == pattern[length]:
                length += 1
                lps[i] = length
                i += 1
            else:
                if length != 0:
                    length = lps[length - 1]
                else:
                    lps[i] = 0
                    i += 1
        return lps

    lps = build_lps(pattern)
    i = j = 0
    while i < len(text):
        if text[i] == pattern[j]:
            i += 1
            j += 1
        if j == len(pattern):
            return i - j  # match found
        elif i < len(text) and text[i] != pattern[j]:
            if j != 0:
                j = lps[j - 1]
            else:
                i += 1
    return -1

In [37]:
# Demo for kmp_search
text = "ababcabcabababd"
pattern = "ababd"
print("Text:", text)
print("Pattern:", pattern)
idx = kmp_search(text, pattern)
print("Pattern found at index:", idx)

Text: ababcabcabababd
Pattern: ababd
Pattern found at index: 10


#### 📌 Reverse a string

In [38]:
def reverse_string(s):
    return s[::-1]

In [39]:
print("balzak reversed: ", reverse_string("balzak"))

balzak reversed:  kazlab


#### 📌 Check if string is palindrome

In [40]:
def is_palindrome(s):
    return s == s[::-1]

In [41]:
print("balzak is palindrome: ", is_palindrome("balzak"))
print("koortsmeetsysteemstrook is palindrome: ", is_palindrome("koortsmeetsysteemstrook"))
print("a man a plan a canal panama is palindrome: ", is_palindrome("a man a plan a canal panama"))

balzak is palindrome:  False
koortsmeetsysteemstrook is palindrome:  True
a man a plan a canal panama is palindrome:  False


#### 📌 Binary search + bisect

**Gebruik:** Snel zoeken of invoegen in een gesorteerde array.  

**Wat gebeurt er:** `bisect_left` geeft positie terug waar een element kan worden toegevoegd zodat de array gesorteerd blijft.

In [42]:
import bisect

arr = sorted([5, 2, 9])
pos = bisect.bisect_left(arr, 7)  # positie om 7 in te voegen

In [43]:
# Demo for bisect_left
print("Sorted array:", arr)
print("Insertion position for 7:", pos)

Sorted array: [2, 5, 9]
Insertion position for 7: 2


## Sorting

#### 📌 Merge Sort

In [44]:
def merge_sort(arr):
    if len(arr) <= 1:
        return arr
    mid = len(arr) // 2
    left = merge_sort(arr[:mid])
    right = merge_sort(arr[mid:])
    res = []
    i = j = 0
    while i < len(left) and j < len(right):
        if left[i] <= right[j]:
            res.append(left[i])
            i += 1
        else:
            res.append(right[j])
            j += 1
    res.extend(left[i:])
    res.extend(right[j:])
    return res

In [45]:
# Demo for quick_sort
arr = [5, 3, 8, 1, 2, 7]
print("Original array:", arr)
print("Sorted array:", merge_sort(arr))

Original array: [5, 3, 8, 1, 2, 7]
Sorted array: [1, 2, 3, 5, 7, 8]


#### 📌 Quick Sort

In [46]:
def quick_sort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[len(arr) // 2]
    left = [x for x in arr if x < pivot]
    mid = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]
    return quick_sort(left) + mid + quick_sort(right)

In [47]:
# Demo for quick_sort
arr = [5, 3, 8, 1, 2, 7]
print("Original array:", arr)
print("Sorted array:", quick_sort(arr))

Original array: [5, 3, 8, 1, 2, 7]
Sorted array: [1, 2, 3, 5, 7, 8]


#### 📌 Counting Sort (integers, O(n+k))

In [48]:
def counting_sort(arr):
    if not arr:
        return []
    max_val = max(arr)
    count = [0] * (max_val + 1)
    for x in arr:
        count[x] += 1
    res = []
    for i, c in enumerate(count):
        res.extend([i] * c)
    return res

In [49]:
# Demo for counting_sort
arr = [5, 3, 8, 1, 2, 7]
print("Original array:", arr)
print("Sorted array:", counting_sort(arr))

Original array: [5, 3, 8, 1, 2, 7]
Sorted array: [1, 2, 3, 5, 7, 8]
